In [83]:
import random
from threading import Thread, Event

In [84]:
class ActionsInterfaceList(list):
    def __init__(self,actions):
        super(ActionsInterfaceList,self).__init__()
        for i,action in enumerate(actions):
            self.append((i,action.public_interface))
    
class Action(object):
    def __init__(self):
        self.text = ""
    
    @property
    def public_interface(self):
        return [self.text,self.is_terminal]
        
    def _callback(self,*args,**kwargs):
        pass
        
    def callback(self,*args,**kwargs):
        return self._callback(*args,**kwargs)

class SubAction(Action):
    is_terminal = False
    def __init__(self):
        super(SubAction,self).__init__()
        
class TermAction(Action):
    is_terminal = True
    def __init__(self):
        super(TermAction,self).__init__()
        
class CardSelectAction(SubAction):
    def __init__(self, card, selected_container):
        super(CardSelectAction, self).__init__()
        self.text = card.text
        self.card = card
        self.selected_container = selected_container
        
    def _callback(self,*args, **kwargs):
        self.selected_container.append(self.card)
            
        

In [85]:
class Card(object):
    def __init__(self,text):
        self.text = text

In [86]:
class Cards(object):
    pass

In [87]:
class GameException(Exception):
    pass

class PlayerException(Exception):
    pass

In [88]:
class Player(object):
    
    def __init__(self):
        self.mana = 0
        self.hero = None
        self.hero_power = None
        self.deck = []
        self.hand = []
        
    def set_deck(self,cards):
        self.deck = cards
        
    def check_ready(self):
        if len(self.deck)<30:
            raise PlayerException("Player has less than 30 cards")
    
    def increase_mana(self):
        if self.mana == 10: return
        self.mana += 1
        
    def draw_card(self):
        new_card = self.deck.pop()
        self.hand.append(new_card)
        
    def first_draw(self,n):
        self.current_draw = [self.deck.pop() for _ in range(n)]
        return self.current_draw
        

In [89]:
class Game(object):
    def __init__(self):
        self.p1 = None
        self.p2 = None
        self.turn = 0
        self._current_player = None
        self.started = False
        self.actions = []
        self.execution_thread = Thread(target=self.run)
        self.execution_thread.daemon = True
        
        self.public_actions_buffer = []
        self.is_waiting_for_action = False
        self.selected_action = None
    
    @property
    def current_player(self):
        if not self._current_player:
            self._current_player = self.p1
        return self._current_player
        
    def add_player(self,player):
        player.check_ready()
        if not self.p1:
            self.p1 = player
            return
        if not self.p2:
            self.p2 = player
            return
        raise GameException("There are already 2 players")
        
    def start(self):
        self.execution_thread.start()
        
    def wait_for_action(self):
        self.is_waiting_for_action = True
        self.pause_event = Event()
        self.pause_event.wait()
        self.is_waiting_for_action = False
        return self.selected_action
    
    def resume(self):
        self.pause_event.set()
        
    def _retrieve_actions(self, actions):
        self.public_actions_buffer = ActionsInterfaceList(actions)
        while True:
            action_id = self.wait_for_action()
            action = actions.pop(action_id)
            self.public_actions_buffer = ActionsInterfaceList(actions)
            action.callback()
            if action.is_terminal:
                break
        
    def run(self):
        self.started = True
        self.init_player()
        self.init_next_turn()
        
    def init_player(self):
        n_cards = 3 if self.current_player == self.p1 else 4
        gets_coin = self.current_player == self.p2
        draw = self.current_player.first_draw(3)
        selected = []
        actions = [CardSelectAction(card,selected) for card in draw] + [TermAction()]
        self._retrieve_actions(actions)
        print selected
        
        
    def switch_player(self):
        self.current_player = self.p1 if self.current_player == self.p2 else self.p2
        
        
    def init_next_turn(self):
        self.current_player.increase_mana()
        self.current_player.draw_card()
        
    def check_valid_action(self,action_id):
        if action_id in [x[0] for x in self.public_actions_buffer]:
            return True
        raise GameException("Invalid action")
    
    def send_action(self,action_id):
        if not self.is_waiting_for_action:
            raise GameException("No action needed")
        self.check_valid_action(action_id)
        self.selected_action = action_id
        self.resume()
        
    def end_turn(self):
        pass
        
    def get_actions(self):
        return self.public_actions_buffer
        

In [90]:
g = Game()

In [91]:
p1 = Player()
p1.set_deck([Card(str(x)) for x in range(30)])

In [92]:
g.add_player(p1)

In [93]:
p2 = Player()
p2.set_deck([Card(str(x+100)) for x in range(30)])

In [94]:
g.start()

In [95]:
g.get_actions()

[(0, ['29', False]), (1, ['28', False]), (2, ['27', False]), (3, ['', True])]

In [96]:
g.send_action(1)

In [97]:
g.get_actions()

[(0, ['29', False]), (1, ['27', False]), (2, ['', True])]

In [98]:
g.is_waiting_for_action

True

In [99]:
result = g.send_action(2)

In [100]:
result[0].text

TypeError: 'NoneType' object has no attribute '__getitem__'